In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import cv2
import os
from matplotlib import pyplot as plt
import os
import cv2
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader,Dataset
import albumentations as albu
from skimage.color import gray2rgb
import functools
import torch
from tqdm.auto import tqdm
import random

import multiprocessing as mp
from multiprocessing.pool import ThreadPool 

In [ ]:
train_csv_path = '/hdd/tam/kaggle/rsna/train.csv'
jpeg_dir = '/hdd/tam/kaggle/rsna/train-jpegs'

In [ ]:
train_df = pd.read_csv(train_csv_path)
train_df.head()

In [ ]:
def get_training_augmentation(y=256,x=256):
    train_transform = [albu.RandomBrightnessContrast(p=0.3),
                           albu.VerticalFlip(p=0.5),
                           albu.HorizontalFlip(p=0.5),
                           albu.Downscale(p=1.0,scale_min=0.35,scale_max=0.75,),
                           albu.Resize(y, x)]
    return albu.Compose(train_transform)


formatted_settings = {
            'input_size': [3, 224, 224],
            'input_range': [0, 1],
            'mean': [0.485, 0.456, 0.406],
            'std': [0.229, 0.224, 0.225],}
def preprocess_input(
    x, mean=None, std=None, input_space="RGB", input_range=None, **kwargs
):

    if input_space == "BGR":
        x = x[..., ::-1].copy()

    if input_range is not None:
        if x.max() > 1 and input_range[1] == 1:
            x = x / 255.0

    if mean is not None:
        mean = np.array(mean)
        x = x - mean

    if std is not None:
        std = np.array(std)
        x = x / std

    return x

def get_preprocessing(preprocessing_fn):
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

def get_validation_augmentation(y=256,x=256):
    """Add paddings to make image shape divisible by 32"""
    test_transform = [albu.Resize(y, x)]
    return albu.Compose(test_transform)

def to_tensor(x, **kwargs):
    """
    Convert image or mask.
    """
    return x.transpose(2, 0, 1).astype('float32')
  

def norm(img):
    img-=img.min()
    return img/img.max()

In [ ]:
class CTDataset2D(Dataset):
    def __init__(self,df,transforms = None,preprocessing=None,size=256,mode='val'):
        df = df[['StudyInstanceUID','SeriesInstanceUID','SOPInstanceUID','pe_present_on_image','negative_exam_for_pe','rightsided_pe','leftsided_pe','central_pe','rv_lv_ratio_gte_1','rv_lv_ratio_lt_1','chronic_pe','acute_and_chronic_pe','indeterminate']]
        self.df_ori = df
        self.df_main = df.values
        if mode=='val':
            self.df = self.df_main
        else:
            self.update_train_df()
#         self.df_group = self.df.groupby(['StudyInstanceUID','SeriesInstanceUID']).agg(list)  
        self.df_label_img = self.df_ori.groupby(['StudyInstanceUID','SeriesInstanceUID','SOPInstanceUID'])['pe_present_on_image'].apply(list)
        self.transforms = transforms
        self.preprocessing = preprocessing
        self.size=size
        self.pool = mp.pool.ThreadPool(16)
    
    def process_one_image(self,image_path):
        img = cv2.imread(image_path)
        if self.transforms:
            img = self.transforms(image=img)['image']
        if self.preprocessing:
            img = self.preprocessing(image=img)['image']
#         imgs.append(img)
        name_split = image_path.split("/")
        row_0 = name_split[-3]
        row_1 = name_split[-2]
        row_2 = name_split[-1].split(".")[0].split("_")[-1]
#         label_imgs.append(self.df_label_img[row_0][row_1][row_2][0])
#         return img, self.df_label_img[row_0][row_1][row_2][0]  
#     def process_label_image(self,image_path):
        
    def __getitem__(self, idx):
        row = self.df[idx]
#         print(idx)
        imgs = []
        label_imgs = []
        list_file = glob.glob(f"{jpeg_dir}/{row[0]}/{row[1]}/*.jpg")
        list_slice = sorted(list_file, key=lambda x : int(x.split("/")[-1].split("_")[0]))
        
        randIndex = random.sample(range(len(list_slice)), 60)
        randIndex.sort()
        list_slice_60 = [list_slice[i] for i in randIndex]
        """
        for i in list_slice_60:
#             print(i)
            img = cv2.imread(i)
            if self.transforms:
#                 print("tran")
                img = self.transforms(image=img)['image']
            if self.preprocessing:
                img = self.preprocessing(image=img)['image']
            imgs.append(img)
            name_split = i.split("/")
            row_0 = name_split[-3]
            row_1 = name_split[-2]
            row_2 = name_split[-1].split(".")[0].split("_")[-1]
            label_imgs.append(self.df_label_img[row_0][row_1][row_2][0])
        """
        results = self.pool.map(self.process_one_image, list_slice_60)
        for img, label in results:
            imgs.append(img)
            label_imgs.append(label)
#         return result

        
        label_scan = row[4:].astype(int)
#         label[2:] = label[2:] if label[0]==1 else 0
        neg_exam = label_scan[0]
        right_side = label_scan[1]
        left_side = label_scan[2]
        central = label_scan[3]
        gte_1 = label_scan[4]
        lt_1 = label_scan[5]
        chronic = label_scan[6]
        acute_and_chronic = label_scan[7]
        acute = 1 - chronic - acute_and_chronic
        indeter = label_scan[8]
        
    
        imgs = np.array(imgs)
        label_imgs = np.array(label_imgs)
        imgs = torch.from_numpy(imgs)
        label_imgs = torch.from_numpy(label_imgs)
        label_scan = np.array([neg_exam,right_side,left_side,central,gte_1,lt_1,chronic,acute_and_chronic,acute,indeter])
        return imgs, label_imgs,torch.from_numpy(label_scan.reshape(-1))

    def __len__(self):
        return len(self.df)
    
    def update_train_df(self):
        df0 = self.df_main[self.df_main[:,3]==0]
        df1 = self.df_main[self.df_main[:,3]==1]
        np.random.shuffle(df0)
        self.df = np.concatenate([df0[:len(df1)],df1],axis=0)

In [ ]:
StudyInstanceUID = list(set(train_df['StudyInstanceUID']))
print(len(StudyInstanceUID))
t_df = train_df[train_df['StudyInstanceUID'].isin(StudyInstanceUID[0:6500])]
v_df = train_df[train_df['StudyInstanceUID'].isin(StudyInstanceUID[6500:])]

In [ ]:
train_dataset.df_label_img['f03743c2432e']['62a77ef402d1']

In [ ]:
train_dataset.df[2]

In [ ]:
t_df[['StudyInstanceUID','SeriesInstanceUID','SOPInstanceUID','pe_present_on_image','negative_exam_for_pe','rightsided_pe','leftsided_pe','central_pe','rv_lv_ratio_gte_1','rv_lv_ratio_lt_1','chronic_pe','acute_and_chronic_pe','indeterminate']]

In [ ]:
t_df

In [ ]:
t_df[t_df['StudyInstanceUID']=='f03743c2432e']['pe_present_on_image'].values

In [ ]:
preprocessing_fn = functools.partial(preprocess_input, **formatted_settings)

train_dataset = CTDataset2D(t_df, preprocessing=get_preprocessing(preprocessing_fn), mode='train')


# train_dataset = CTDataset2D(t_df,transforms=get_training_augmentation(), preprocessing=get_preprocessing(preprocessing_fn), mode='train')

In [ ]:
aa = train_dataset.__getitem__(1)

In [ ]:
aa[0]

In [ ]:
aa[0].shape

In [ ]:
aa[2]

In [ ]:
train_dataloader = DataLoader(train_dataset,
                             shuffle=True,
                             batch_size=4,
                             num_workers=2)

In [ ]:
ll = iter(train_dataloader)

In [ ]:
data = next(ll)

In [ ]:
data[0].shape

In [ ]:
from torchvision.models.resnet import resnet18,resnext50_32x4d
from torch import nn, optim


class RsnaImageModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        
#         self.backbone = resnext50_32x4d(pretrained=True)
        self.backbone = resnet18 (pretrained=False)
        

        self.backbone.conv1 = nn.Conv2d(
            3,
            self.backbone.conv1.out_channels,
            kernel_size=self.backbone.conv1.kernel_size,
            stride=self.backbone.conv1.stride,
            padding=self.backbone.conv1.padding,
            bias=False,
        )
        
        # This is 512 for resnet18 and resnet34;
        # And it is 2048 for the other resnets
        backbone_out_features = 512
        # X, Y coords for the future positions (output shape: Bx50x2)
        # You can add more layers here.
        self.head = nn.Sequential(
            # nn.Dropout(0.2),
            nn.Linear(in_features=backbone_out_features, out_features=4096),
        )
        self.logit = nn.Linear(4096, out_features=512)
        
    def forward(self, xs):
        x = self.backbone.conv1(xs[0])
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)
        x = self.backbone.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.head(x)
        x = self.logit(x)
#         print(x.shape)
        out = torch.unsqueeze(x,0)
        for i in range(1,len(xs)):
            x = self.backbone.conv1(xs[i])
            x = self.backbone.bn1(x)
            x = self.backbone.relu(x)
            x = self.backbone.maxpool(x)

            x = self.backbone.layer1(x)
            x = self.backbone.layer2(x)
            x = self.backbone.layer3(x)
            x = self.backbone.layer4(x)
            x = self.backbone.avgpool(x)
            x = torch.flatten(x, 1)
            x = self.head(x)
            x = self.logit(x)
            x = torch.unsqueeze(x,0)
            
            out = torch.cat((out,x),0)
            
        return out

In [ ]:
class RsnaLSTMModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_sz  = 512 #(2000 from fcn_en_output reshape to 50*40)
        self.hidden_sz = 1024
#         self.hidden_sz_en = 128
        self.num_layer = 10
        
        self.interlayer = 512
        
        self.encoderLSTM = nn.LSTM(self.input_sz,self.hidden_sz,self.num_layer,batch_first=True)
#         self.Decoder_lstm = nn.LSTM( self.input_sz,self.hidden_sz,self.num_layer,batch_first=True)

        
        self.fcn_en_state_dec_state= nn.Sequential(nn.Linear(in_features=self.hidden_sz, out_features=self.interlayer),
                            nn.ReLU(inplace=True),
                            nn.Linear(in_features=self.interlayer, out_features=1024)
                            )


    def forward(self, embeds):
        out,(hn,cn) = self.encoderLSTM(embeds)
#         print(x.shape)
        x = out[:,-1,:]
        x = self.fcn_en_state_dec_state(x)
#         x = torch.flatten(x, 1)
        return x

In [ ]:
from torchsummaryX import summary

In [ ]:
lstm = RsnaLSTMModel()

In [ ]:
# summary(lstm,(60,512))

In [ ]:
lstm

In [ ]:
llll=lstm(o)

In [ ]:
llll.shape

In [ ]:
llll

In [ ]:
xxx = nn.Sequential(nn.Linear(in_features=1024, out_features=2),
                                      nn.Softmax(dim=2))

In [ ]:
# xxx(llll)

In [ ]:
class RsnaFullModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        
#         self.backbone = resnext50_32x4d(pretrained=True)
        self.image_model = RsnaImageModel()
        self.lstm_model = RsnaLSTMModel()
        self.image_feature = 512
        self.study_feature = 1024
        self.logit_image = nn.Sequential(nn.Linear(in_features=self.image_feature, out_features=1),
                                      nn.Sigmoid())
        
        self.neg_exam = nn.Sequential(nn.Linear(in_features=self.study_feature, out_features=1),
                                      nn.Sigmoid())
        self.side_PE = nn.Sequential(nn.Linear(in_features=self.study_feature, out_features=3),
                                      nn.Softmax(dim=1))
        self.ratio = nn.Sequential(nn.Linear(in_features=self.study_feature, out_features=2),
                                      nn.Softmax(dim=1))
        self.type_PE = nn.Sequential(nn.Linear(in_features=self.study_feature, out_features=3),
                                      nn.Softmax(dim=1))
        self.QA = nn.Sequential(nn.Linear(in_features=self.study_feature, out_features=2),
                                      nn.Softmax(dim=1))
    def forward(self, xs):
        img_feature = self.image_model(xs)
        study_out = self.lstm_model(img_feature)
        img_out = self.logit_image(img_feature)   
        
        neg_exam_out = self.neg_exam(study_out)
        side_PE_out = self.side_PE(study_out)
        ratio_out = self.ratio(study_out)
        type_PE_out = self.type_PE(study_out)
#         QA = self.QA(study_out)
        
        scan_out = torch.cat([neg_exam_out,side_PE_out,ratio_out,type_PE_out],1)
        return img_out,scan_out

In [ ]:
model_full = RsnaFullModel()

In [ ]:
model = RsnaImageModel()

In [ ]:
from torchsummary import summary

In [ ]:
summary(model,(60,3,256,256))

In [ ]:
o = model(data[0])

In [ ]:
o

In [ ]:
inp = data[0]
re = model_full(inp)

In [ ]:
re[0].shape

In [ ]:
del model_full
del re
del inp

In [ ]:
re[0].shape

In [ ]:
re[1].shape

In [ ]:
del model_full

In [ ]:
torch.cuda.empty_cache()

In [ ]:
torch.cuda.empty()

In [ ]:
classes = ['{}_pe_present_on_image',
 '{}_negative_exam_for_pe',
 '{}_qa_motion',
 '{}_qa_contrast',
 '{}_flow_artifact',
 '{}_rv_lv_ratio_gte_1',
 '{}_rv_lv_ratio_lt_1',
 '{}_leftsided_pe',
 '{}_chronic_pe',
 '{}_true_filling_defect_not_pe',
 '{}_rightsided_pe',
 '{}_acute_and_chronic_pe',
 '{}_central_pe',
 '{}_indeterminate']